# Imports

In [6]:
from numpy.random import seed
seed(888)
from tensorflow.compat.v1 import set_random_seed
set_random_seed(404)

In [7]:
import os
import numpy as np
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

# Constants

In [8]:
x_train_path='handwriting/digit_xtrain.csv'
y_train_path='handwriting/digit_ytrain.csv'
x_test_path='handwriting/digit_xtest.csv'
y_test_path='handwriting/digit_ytest.csv'
nr_classes=10
valid_size=10000
total_inputs=28*28*1 # image height * width * channels

logging_path='tb_mnist_logs/'

# Get the Data

In [9]:
%%time
x_train_all=np.loadtxt(x_train_path, delimiter=',', dtype=int)

Wall time: 20.9 s


In [10]:
%%time
x_test=np.loadtxt(x_test_path, delimiter=',', dtype=int)
y_train_all=np.loadtxt(y_train_path, delimiter=',', dtype=int)
y_test=np.loadtxt(y_test_path, delimiter=',', dtype=int)

Wall time: 3.87 s


# Preprocessing Data

## Rescale the data to from 255 to 1.0

In [11]:
x_train_all, x_test = x_train_all/255.0 , x_test/ 255.0

## Convert our Y set lables to One Hot Encodeing with numpy eye()

In [12]:
y_train_all= np.eye(nr_classes)[y_train_all]

In [13]:
y_test=np.eye(nr_classes)[y_test]

In [14]:
y_train_all.shape

(60000, 10)

### make validation sets from trainig datasets 

In [15]:
x_val=x_train_all[:valid_size]
y_val=y_train_all[:valid_size]

In [16]:
x_train=x_train_all[valid_size:]
y_train=y_train_all[valid_size:]

In [17]:
x_train.shape

(50000, 784)

In [18]:
x_val.shape

(10000, 784)

# Setting and Creating Tensors

In [111]:
x=tf.placeholder(tf.float32, shape=[None, total_inputs],name="X")
y=tf.placeholder(tf.float32, shape=[None, nr_classes],name="labels")


## Neural Network Architcture

### defining our Hyperparams

In [112]:
nr_epochs= 50
learning_rate =1e-3
n_hidden1=512
n_hidden2=64

## Initialization 

### first Layer

In [113]:
# with tf.name_scope('Hidden_1'):
#     init_w1=tf.compat.v1.truncated_normal(shape=[total_inputs, n_hidden1], stddev=0.1, seed=42)
#     w1= tf.Variable(init_w1,name='W1')
#     init_b1=tf.constant(value=0.0, shape=[n_hidden1])
#     b1=tf.Variable(init_b1,name='b1')
#     layer1_in = tf.matmul(x, w1)+ b1
#     layer1_out= tf.nn.relu(layer1_in)


### second Layer

In [114]:
# with tf.name_scope('Hidden_2'):
#     init_w2=tf.compat.v1.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
#     w2= tf.Variable(init_w2,name='W2')
#     init_b2=tf.constant(value=0.0, shape=[n_hidden2])
#     b2=tf.Variable(init_b2,name='b2')
#     layer2_in = tf.matmul(layer1_out, w2)+ b2
#     layer2_out= tf.nn.relu(layer2_in)

### output Layer

In [115]:
# with tf.name_scope('Output_Layer'):
#     init_w3=tf.compat.v1.truncated_normal(shape=[n_hidden2, nr_classes], stddev=0.1, seed=42)
#     w3= tf.Variable(initial_value=init_w3,name='W3')
#     init_b3=tf.constant(value=0.0, shape=[nr_classes])
#     b3=tf.Variable(init_b3,name='b3')
#     layer3_in = tf.matmul(layer2_out, w3)+ b3
#     output= tf.nn.softmax(layer3_in)    

# Automate to create the Layers:

In [116]:
def setup_layer(input,weight_dim, bias_dim, name):
    with tf.name_scope(name):

        init_w=tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w= tf.Variable(initial_value=init_w,name='W')

        init_b=tf.constant(value=0.0, shape=bias_dim)
        b=tf.Variable(initial_value=init_b,name='B')

        layer_in = tf.matmul(input, w)+ b

        if (name=='out'):
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
        
        tf.summary.histogram('weights',w)
        tf.summary.histogram('biases',b)

        return layer_out

## Setup the layer with the Function:

In [117]:
# layer_1=setup_layer(x,weight_dim=[total_inputs,n_hidden1],bias_dim=[n_hidden1],name='layer_1')

# layer_2=setup_layer(layer_1,weight_dim=[n_hidden1,n_hidden2],bias_dim=[n_hidden2],name='layer_2')

# output=setup_layer(layer_2,weight_dim=[n_hidden2,nr_classes],bias_dim=[nr_classes],name='out')

In [118]:
layer_1=setup_layer(x,weight_dim=[total_inputs,n_hidden1],bias_dim=[n_hidden1],name='layer_1')

layer_drop=tf.nn.dropout(layer_1,keep_prob=0.8, name='layer_drop')

layer_2=setup_layer(layer_drop,weight_dim=[n_hidden1,n_hidden2],bias_dim=[n_hidden2],name='layer_2')

output=setup_layer(layer_2,weight_dim=[n_hidden2,nr_classes],bias_dim=[nr_classes],name='out')

# Setup Tensorboard

In [119]:
from time import strftime

In [120]:
# Create the folder to be used with Tensorboard
folder_name=f'{n_hidden1}-{n_hidden2}-D- at {strftime("%H_%M")}'
directory=os.path.join(logging_path,folder_name)
try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print("dirs created successfully")


dirs created successfully


In [121]:
# folder_name=f'Model 1 at {strftime("%H:%M")}'
# folder_name

# Setup Loss, Optimizer, Metrics

### Loss

In [122]:
with tf.name_scope('Loss'):
    loss= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=output))

### Optimizer

In [123]:
with tf.name_scope('optimizer'):
    optimizer=tf.train.AdamOptimizer(learning_rate)
    train_step=optimizer.minimize(loss)

### Accuracy Metric

In [124]:
with tf.name_scope('accuracy_metrics'):    
    correct_pred=tf.equal(tf.argmax(output, axis=1),tf.argmax(y,axis=1))
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [125]:
with tf.name_scope('performance'):
    tf.compat.v1.summary.scalar('accuracy', accuracy)
    tf.compat.v1.summary.scalar('cost', loss)


In [126]:
with tf.name_scope('showImage'):
    x_image=tf.reshape(x, [-1, 28,28,1])
    tf.summary.image('image_input',x_image,max_outputs=10)

# Run Session

In [127]:
sess=tf.Session()

In [128]:
# initialisation all the Variables
init=tf.global_variables_initializer()
sess.run(init)

## Create Filewriter:

In [129]:
merage_summary=tf.summary.merge_all()
train_writer=tf.summary.FileWriter(directory+'/train')
train_writer.add_graph(sess.graph)

validation_writer=tf.summary.FileWriter(directory+'/validation')

## Batching the Data

In [130]:
size_of_batch=1000
num_examples=y_train.shape[0]
nr_iterations=int(num_examples/size_of_batch)
index_in_epoch=0

In [131]:
nr_iterations

50

### Build Function to help batching the Data:

In [132]:
def next_batch(batch_size, data, labels):
    global num_examples
    global index_in_epoch
    # if we reach the end of the Dataset
    start=index_in_epoch
    index_in_epoch += batch_size
    if index_in_epoch > num_examples:
        start=0
        index_in_epoch=batch_size

    end=index_in_epoch

    return data[start:end] , labels[start:end]

# Trainig Loop:

In [133]:
for epoch in range(nr_epochs):
    for i in range(nr_iterations):
        batch_x, batch_y= next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)

        feed_dictionary= {x:batch_x, y:batch_y}
        sess.run(train_step, feed_dict=feed_dictionary)

    s,batch_accuracy=sess.run(fetches=[merage_summary,accuracy], feed_dict=feed_dictionary)
    train_writer.add_summary(s,epoch)
    print(f'Epoch {epoch} \t| Trainig Accuracy {batch_accuracy}')
    # ====================== Validation 
    summary=sess.run(fetches=merage_summary, feed_dict={x:x_val, y:y_val})
    validation_writer.add_summary(summary,epoch)

print("Training Done!")

Epoch 0 	| Trainig Accuracy 0.8489999771118164
Epoch 1 	| Trainig Accuracy 0.859000027179718
Epoch 2 	| Trainig Accuracy 0.8659999966621399
Epoch 3 	| Trainig Accuracy 0.8690000176429749
Epoch 4 	| Trainig Accuracy 0.8740000128746033
Epoch 5 	| Trainig Accuracy 0.875
Epoch 6 	| Trainig Accuracy 0.972000002861023
Epoch 7 	| Trainig Accuracy 0.9789999723434448
Epoch 8 	| Trainig Accuracy 0.9779999852180481
Epoch 9 	| Trainig Accuracy 0.9819999933242798
Epoch 10 	| Trainig Accuracy 0.9829999804496765
Epoch 11 	| Trainig Accuracy 0.9829999804496765
Epoch 12 	| Trainig Accuracy 0.984000027179718
Epoch 13 	| Trainig Accuracy 0.9879999756813049
Epoch 14 	| Trainig Accuracy 0.9850000143051147
Epoch 15 	| Trainig Accuracy 0.9890000224113464
Epoch 16 	| Trainig Accuracy 0.9890000224113464
Epoch 17 	| Trainig Accuracy 0.9900000095367432
Epoch 18 	| Trainig Accuracy 0.9900000095367432
Epoch 19 	| Trainig Accuracy 0.9890000224113464
Epoch 20 	| Trainig Accuracy 0.984000027179718
Epoch 21 	| Trainig

# Evaluating:

In [134]:
from PIL import Image

## using pil to load our test_img

In [135]:
img= Image.open('handwriting/test_img.png')
img

## convert the img  black_wight mode then invert then colors then to array then flat it

In [136]:
bw=img.convert('L')
img_array=np.invert(bw)
img_array.shape
test_img=img_array.ravel()
test_img.shape
prediction= sess.run(fetches=tf.argmax(output,axis=1), feed_dict={x:[test_img]})

In [137]:
print(f'Prediction for test image is {prediction}')

Prediction for test image is [2]


In [138]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={x:x_test, y:y_test})
print(f'Accuracy on test set is {test_accuracy:0.2%}')

Accuracy on test set is 98.04%


In [139]:
saver = tf.train.Saver()
saver.save(sess, 'my-test-model')

'my-test-model'

In [140]:

train_writer.close()
validation_writer.close()
sess.close()
tf.compat.v1.reset_default_graph()